In [6]:
import numpy as np

m=np.random.randint(40,50)
n=np.random.randint(200,300)

from scipy import linalg,sparse
x=sparse.random(m,n,density=0.05,format='csr',dtype='float')*100
mat=np.round(x.todense(),decimals=0)

#remove void rows and columns
rs=np.sum(mat,axis=1)
cs=np.sum(mat,axis=0)
rows=np.where(rs!=0)[0]
rows=np.append(rows,m-1)
cols=np.where(cs!=0)[1]
mat=mat[:,cols][rows,:]
 
rs=np.sum(mat,axis=1)
cs=np.sum(mat,axis=0)

#create supply demand
demands= np.array(mat.sum(axis=1)).T[0]
supplies=np.array(mat.sum(axis=0))[0]
sparse=np.matrix(np.copy(mat), dtype = int)
sparse[sparse>0]=1

# sparse=np.matrix([[0, 0, 0, 1, 0, 1, 0, 1],
#        [0, 0, 0, 1, 1, 1, 1, 1],
#        [0, 0, 1, 0, 1, 1, 0, 0],
#        [1, 0, 0, 0, 0, 0, 0, 0],
#        [1, 1, 1, 0, 0, 0, 0, 0],
#        [1, 1, 1, 1, 1, 1, 1, 1]])

# supplies=np.reshape(np.array([92,32,93,83,78,138,55,75], dtype = int), 8)
# demands=np.array([96,149,90,28,81,202])

demands, supplies

(array([  340.,  1020.,   857.,   906.,   386.,   644.,   592.,   610.,
          561.,   455.,   714.,   722.,   669.,   646.,   870.,   859.,
          668.,   960.,   863.,   857.,  1030.,   412.,   857.,   652.,
          838.,   588.,   632.,   737.,   239.,   369.,   440.,   370.,
          395.,   168.,   722.,   243.,   878.,   450.,   298.,   670.,
          620.,   661.,   705.,   581.,   797.,   720.,   720.]),
 array([ 116.,  116.,  152.,   56.,   56.,  147.,  283.,  116.,   31.,
          99.,  118.,  213.,  119.,  183.,   75.,   56.,   54.,  131.,
          48.,   83.,  192.,   46.,  151.,  260.,  131.,   40.,  165.,
          26.,   91.,  115.,   34.,  112.,  160.,  260.,  192.,  123.,
         135.,  149.,   74.,  222.,   73.,  137.,   86.,   58.,  141.,
          83.,   97.,   30.,  217.,  304.,  113.,   64.,  206.,   46.,
         264.,  171.,   97.,  151.,  305.,   76.,   82.,   96.,    7.,
         172.,  110.,   52.,  136.,  172.,  105.,  154.,   75.,  183.,
      

In [7]:
## List of utility functions

def get_intersections_with_existing_constraints(current_constraint, existing_constraints):
    return [current_constraint & existing_constraints[i] for i in range(len(existing_constraints))]

def get_all_subsets(current_constraint, intersections_with_existing_constraints, existing_constraints):
    all_subsets = [i for i, j in zip(intersections_with_existing_constraints, existing_constraints) if i == j]
    all_subsets_minus_self = [x for x in all_subsets if x != current_constraint]
    return all_subsets_minus_self

def get_all_super_subsets(subset_list):
    return [i for i in subset_list if i not in [i & j for j in subset_list if i != j]]

def get_all_supersets(current_constraint, intersections_with_existing_constraints, existing_constraints):
    all_supersets = [existing_constraints[i] for i in range(len(existing_constraints)) if intersections_with_existing_constraints[i] == current_constraint]
    all_supersets_minus_self = [x for x in all_supersets if x != current_constraint]
    return all_supersets_minus_self

def get_all_overlaps(current_constraint, intersections_with_existing_constraints, existing_constraints):
    all_overlaps = [existing_constraints[i] for i in range(len(existing_constraints)) if bool(intersections_with_existing_constraints[i])]
    all_overlaps_minus_self = [x for x in all_overlaps if x != current_constraint]
    return all_overlaps_minus_self

def get_non_sub_super_overlaps(all_overlaps, all_subsets, all_supersets):
    combined_sub_super_sets = all_subsets + all_supersets
    filtered_overlaps = [x for x in all_overlaps if x not in combined_sub_super_sets]
    return filtered_overlaps

def get_overlaps_with_unique_supersets(all_overlaps, current_constraint):
    removal_set = []
    for element1 in all_overlaps:
        for element2 in all_overlaps:
            if element1 & element2 == element1 and element1 != element2: 
                ''' An assumption here as well: we need only those overlaps which are not subset of any other overlap'''
                #element1 is subset of element2; they are not equal and both have same superset
                removal_set.append(element1)
    filtered_overlaps = [element for element in all_overlaps if element not in removal_set]
    return filtered_overlaps

def modify_constraint(current_superset, current_demand, existing_constraints, constraints_demand, constraints_availability):
    constraint_index = existing_constraints.index(current_superset)
    constraints_demand[constraint_index] = constraints_demand[constraint_index] + current_demand
    constraints_availability[constraint_index] = constraints_availability[constraint_index] - current_demand
    return constraints_demand, constraints_availability

def add_constraint(current_request, current_demand, current_supply, existing_constraints, constraints_supply, constraints_demand, constraints_availability):
    existing_constraints.append(current_request)
    constraints_supply.append(current_supply)
    constraints_demand.append(current_demand)
    constraints_availability.append(current_supply - current_demand)
    return existing_constraints, constraints_supply, constraints_demand, constraints_availability

def remove_redundancy_type_1(existing_constraints, constraints_supply, constraints_demand, constraints_availability):
    ''' Redundancy type 1 means when a subset is less or equally restrictive than a superset.
        Example: "x1 + x2 <= 20" is redundant if we also have "x1 + x2 + x3 <= 20" '''
    
    removal_list = []
    for current_row in range(len(existing_constraints)):
        current_constraint = existing_constraints[current_row]
        current_availability = constraints_availability[current_row]
        intersection_list = get_intersections_with_existing_constraints(current_constraint, existing_constraints)        
        all_supersets = get_all_supersets(current_constraint, intersection_list, existing_constraints)
        for current_superset in all_supersets:
            superset_availability = constraints_availability[existing_constraints.index(current_superset)]
            if superset_availability <= current_availability:
                removal_list.append(current_row)
                break
    existing_constraints = [existing_constraints[i] for i in range(len(existing_constraints)) if i not in removal_list]
    constraints_supply = [constraints_supply[i] for i in range(len(constraints_supply)) if i not in removal_list]
    constraints_demand = [constraints_demand[i] for i in range(len(constraints_demand)) if i not in removal_list]
    constraints_availability = [constraints_availability[i] for i in range(len(constraints_availability)) if i not in removal_list]
    return existing_constraints, constraints_supply, constraints_demand, constraints_availability

def remove_redundancy_type_2(existing_constraints, constraints_supply, constraints_demand, constraints_availability):
    ''' Redundancy type 2 means when sum of disjoint super subsets is less than a superset, the superset is redundant.
        This is true only if disjoint super subsets together can constitute the superset.
        Example: "x1 + x2 + x3 <= 20" is redundant if we also have "x1 + x2 <= 10" and x3 <=5 '''
    
    removal_list = []
    for current_row in range(len(existing_constraints)):
        current_constraint = existing_constraints[current_row]
        current_availability = constraints_availability[current_row]
        intersection_list = get_intersections_with_existing_constraints(current_constraint, existing_constraints)        
        all_subsets = get_all_subsets(current_constraint, intersection_list, existing_constraints)
        all_super_subsets = get_all_super_subsets(all_subsets)
        if len(all_super_subsets) > 0:
            union_of_super_subsets = set.union(*all_super_subsets)
            if union_of_super_subsets == current_constraint:
                total_availability_of_subsets = 0
                for subset in all_super_subsets:
                    total_availability_of_subsets = total_availability_of_subsets + constraints_availability[existing_constraints.index(subset)]
                if total_availability_of_subsets <= current_availability:
                    removal_list.append(current_row)
    
    existing_constraints = [existing_constraints[i] for i in range(len(existing_constraints)) if i not in removal_list]
    constraints_supply = [constraints_supply[i] for i in range(len(constraints_supply)) if i not in removal_list]
    constraints_demand = [constraints_demand[i] for i in range(len(constraints_demand)) if i not in removal_list]
    constraints_availability = [constraints_availability[i] for i in range(len(constraints_availability)) if i not in removal_list]
    return existing_constraints, constraints_supply, constraints_demand, constraints_availability


In [8]:
## The main logic of constraint creation and modifications

def update_constraints_list(current_request, current_demand, existing_constraints, constraints_supply, constraints_demand, constraints_availability, supplies):
    
    intersection_list = get_intersections_with_existing_constraints(current_request, existing_constraints)
    all_subsets = get_all_subsets(current_request, intersection_list, existing_constraints)
    all_subsets = get_all_super_subsets(all_subsets)
    all_supersets = get_all_supersets(current_request, intersection_list, existing_constraints)
    all_overlaps = get_all_overlaps(current_request, intersection_list, existing_constraints)
    all_overlaps = get_non_sub_super_overlaps(all_overlaps, all_subsets, all_supersets)
    all_overlaps = get_overlaps_with_unique_supersets(all_overlaps, current_request)
    
    if current_request in existing_constraints:
        ## constraint with same set already exists
        all_supersets.append(current_request)
        for current_superset in all_supersets:
            constraints_demand, constraints_availability = modify_constraint(current_superset, current_demand, existing_constraints, constraints_demand, constraints_availability)
    
    elif len(all_subsets) > 0 or len(all_supersets) > 0 or len(all_overlaps) > 0:
        ## constraint with same set doesn't exist but subsets/supersets exist
        ## Update the supersets first
        for current_superset in all_supersets:
            constraints_demand, constraints_availability = modify_constraint(current_superset, current_demand, existing_constraints, constraints_demand, constraints_availability)
        
        ## Update current constraint based on disjoint subsets and add it to constraints list
        '''We take here only super subsets. These subsets don't have any of their subsets in this list. 
        This also ensures that two elements in this list will have absolutely zero intersection/overlaps. Had there been
        any overlap, their superset would have also existed in the existing_constraints and this superset would have got 
        selected instead of these two sets.'''
        current_demand_including_subset = current_demand
        for current_subset in all_subsets:
            subset_index = existing_constraints.index(current_subset)
            current_demand_including_subset = current_demand_including_subset + constraints_demand[subset_index]
        current_supply = sum([supplies[i] for i in list(current_request)])
        existing_constraints, constraints_supply, constraints_demand, constraints_availability = add_constraint(current_request, current_demand_including_subset, current_supply, existing_constraints, constraints_supply, constraints_demand, constraints_availability)
        
        ## For oevrlap list keep creating new supersets and add them; skip if such superset already exist
        if len(all_overlaps) > 0:
            for current_overlap in all_overlaps:
                new_superset = current_request | current_overlap
                if new_superset in existing_constraints:
                    continue
                else:
                    overlap_index = existing_constraints.index(current_overlap)
                    new_superset_demand = current_demand_including_subset + constraints_demand[overlap_index] #add current_B
                    new_superset_supply = sum([supplies[i] for i in list(new_superset)])
                    new_superset_RHS = new_superset_supply - new_superset_demand #reduce from new_supply
                    existing_constraints, constraints_supply, constraints_demand, constraints_availability = add_constraint(new_superset, new_superset_demand, new_superset_supply, existing_constraints, constraints_supply, constraints_demand, constraints_availability)
                    ## exit
    
    else:
        ## It is from different island
        ## add current constraint
        current_supply = sum([supplies[i] for i in list(current_request)])
        existing_constraints, constraints_supply, constraints_demand, constraints_availability = add_constraint(current_request, current_demand, current_supply, existing_constraints, constraints_supply, constraints_demand, constraints_availability)
        ## exit
    
    existing_constraints, constraints_supply, constraints_demand, constraints_availability = remove_redundancy_type_1(existing_constraints, constraints_supply, constraints_demand, constraints_availability)
    existing_constraints, constraints_supply, constraints_demand, constraints_availability = remove_redundancy_type_2(existing_constraints, constraints_supply, constraints_demand, constraints_availability)

    return existing_constraints, constraints_supply, constraints_demand, constraints_availability
    

In [9]:
## Main program

import datetime
start_time = datetime.datetime.now()

## converting everything to lists
request_list = []
for i in range(sparse.shape[0]):
    request_list.append(set(np.where(sparse[i,])[1]))
supplies = list(supplies)
demands = list(demands)

## Adding first constraint
existing_constraints = [request_list[0]]
constraints_supply = [sum([supplies[i] for i in list(existing_constraints[0])])]
constraints_demand = [demands[0]]
constraints_availability = [constraints_supply[0] - constraints_demand[0]]

# Looping for other rows
for current_row in range(1, (len(request_list)-1)):
    current_request = request_list[current_row]
    current_demand = demands[current_row]
    existing_constraints, constraints_supply, constraints_demand, constraints_availability = update_constraints_list(current_request, current_demand, existing_constraints, constraints_supply, constraints_demand, constraints_availability, supplies)
    print(current_row, len(existing_constraints), len(constraints_availability))
    
# end_time = datetime.datetime.now()
existing_constraints, constraints_availability

end_time = datetime.datetime.now()

1 3 3
2 5 5
3 7 7
4 9 9
5 11 11
6 13 13
7 15 15
8 17 17
9 19 19
10 21 21
11 23 23
12 25 25
13 27 27
14 29 29
15 28 28
16 30 30
17 31 31
18 30 30
19 31 31
20 30 30
21 31 31
22 31 31
23 32 32
24 32 32
25 33 33
26 33 33
27 34 34
28 35 35
29 35 35
30 36 36
31 37 37
32 38 38
33 45 45
34 44 44
35 45 45
36 46 46
37 64 64
38 64 64
39 65 65
40 65 65
41 65 65
42 55 55
43 45 45
44 16 16
45 3 3


In [10]:
end_time - start_time

datetime.timedelta(0, 0, 183483)